In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes

In [2]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run_output")

In [3]:
# Change this line only when you receive a new run
run_folder = "sisepuede_run_2025-07-25T12;55;12.325978"
file_name  = f"{run_folder}.csv"
file_name

'sisepuede_run_2025-07-25T12;55;12.325978.csv'

In [4]:
# Define the run folder and tableau data path
run = output_data.joinpath(run_folder)
tableau = dir_cur.joinpath("Tableau/data")

In [5]:
# Define the region
region = "costa_rica"

In [6]:
# Load the output data
ssp_output = pd.read_csv(run.joinpath(file_name))
ssp_output.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,0,costa_rica,0,0.0,367983.831774,68239.857458,80.234375,79198.541655,6714.566582,1.154590e+06,...,20.037589,12.059233,2.946800,6.177415,0.0,2.755621,8.636027,73.140598,37.547100,28.821448
1,0,costa_rica,1,0.0,362385.636787,67201.713944,79.013757,77993.681983,6612.416842,1.137025e+06,...,20.479993,12.539800,3.455483,7.165802,0.0,3.259699,8.228317,71.163825,39.000100,29.765171
2,0,costa_rica,2,0.0,361096.126663,66962.583908,78.732596,77716.149895,6588.887271,1.132979e+06,...,19.994821,12.161233,3.041817,11.119348,0.0,3.958685,8.302446,89.448975,39.119475,28.720595
3,0,costa_rica,3,0.0,359935.747562,66747.400257,78.479589,77466.409758,6567.713942,1.129338e+06,...,18.948929,12.289167,3.375283,5.436126,0.0,4.167037,8.846059,90.190264,37.352525,29.482348
4,0,costa_rica,4,0.0,358773.521179,66531.874041,78.226180,77216.272044,6546.506906,1.125692e+06,...,20.238607,12.337700,2.992183,6.177415,0.0,3.226093,7.882382,84.012849,38.551850,32.769776


In [7]:
# Load atribute primary data
df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
df_primary.head()

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,1001,0,1000,0


In [8]:
# Load atribute strategy data
df_strategy =pd.read_csv(run.joinpath("ATTRIBUTE_STRATEGY.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
64,4005,IPPU:DEC_PFCS,Singleton - Default Value - IPPU: Reduce use o...,NaN,TX:IPPU:DEC_PFCS,0
65,4006,IP:ALL,Sectoral Composite - IPPU,All (unique by transformer) IPPU transformations,TX:IPPU:DEC_CLINKER|TX:IPPU:DEC_DEMAND|TX:IPPU...,0
66,6000,PFLO:INC_HEALTHIER_DIETS,Singleton - Default Value - PFLO: Change diets,NaN,TX:PFLO:INC_HEALTHIER_DIETS,0
67,6001,PFLO:INC_IND_CCS,Singleton - Default Value - PFLO: Industrial c...,NaN,TX:PFLO:INC_IND_CCS,0
68,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0


In [9]:
# Merge the dataframes to include primary and strategy information in the ssp_output
ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(  df_strategy, how='left', on='strategy_id')

In [10]:
# Strategys in the output
ssp_output.value_counts('strategy')

strategy
Singleton - Default Value - AGRC: Improve rice management    36
Strategy TX:BASE                                             36
Name: count, dtype: int64

In [11]:
# Get the list of all fields in the output
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [12]:
# Define subsector emission fields that are not energy related
subsector_emission_fields_non_energy = [f for f in subsector_emission_fields if f not in ['emission_co2e_subsector_total_inen', 'emission_co2e_subsector_total_fgtv', 'emission_co2e_subsector_total_entc']]
subsector_emission_fields_non_energy

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [13]:
# Define year based on time_period
ssp_output['year'] = ssp_output['time_period'] + 2015

In [14]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields_non_energy,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,emission_co2e_subsector_total_agrc,1.584478
1,0,Strategy TX:BASE,0,2016,emission_co2e_subsector_total_agrc,1.573848
2,0,Strategy TX:BASE,0,2017,emission_co2e_subsector_total_agrc,1.667754
3,0,Strategy TX:BASE,0,2018,emission_co2e_subsector_total_agrc,1.557793
4,0,Strategy TX:BASE,0,2019,emission_co2e_subsector_total_agrc,1.538893


In [15]:
# Clean the 'Subsector' column to remove the prefix and capitalize subsector names
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agrc,1.584478
1,0,Strategy TX:BASE,0,2016,Agrc,1.573848
2,0,Strategy TX:BASE,0,2017,Agrc,1.667754
3,0,Strategy TX:BASE,0,2018,Agrc,1.557793
4,0,Strategy TX:BASE,0,2019,Agrc,1.538893


In [16]:
# List of subsectors in the output
ssp_long.value_counts('Subsector')

Subsector
Agrc    72
Ccsq    72
Frst    72
Ippu    72
Lndu    72
Lsmm    72
Lvst    72
Scoe    72
Soil    72
Trns    72
Trww    72
Waso    72
Name: count, dtype: int64

In [17]:
# Define a dictionary to map subsector codes to full names
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               72
Carbon Capture and Sequestration          72
Forest                                    72
IPPU                                      72
Land Use                                  72
Livestock                                 72
Livestock Manure Management               72
Soil Management                           72
Solid Waste                               72
Stationary Combustion and Other Energy    72
Transportation                            72
Wastewater Treatment                      72
Name: count, dtype: int64

In [18]:
# Rename columns
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agriculture,1.584478
1,0,Strategy TX:BASE,0,2016,Agriculture,1.573848
2,0,Strategy TX:BASE,0,2017,Agriculture,1.667754
3,0,Strategy TX:BASE,0,2018,Agriculture,1.557793
4,0,Strategy TX:BASE,0,2019,Agriculture,1.538893
...,...,...,...,...,...,...
859,1001,Singleton - Default Value - AGRC: Improve rice...,1000,2046,IPPU,111.786446
860,1001,Singleton - Default Value - AGRC: Improve rice...,1000,2047,IPPU,113.649990
861,1001,Singleton - Default Value - AGRC: Improve rice...,1000,2048,IPPU,115.642667
862,1001,Singleton - Default Value - AGRC: Improve rice...,1000,2049,IPPU,117.768492


In [20]:
# Save the long format data to a CSV file for Tableau
ssp_long.to_csv(tableau.joinpath(f"emissions_{region}_{file_name}.csv"), index=False)